In [1]:
import os
import torch
import random
import pprint
import numpy as np
from loguru import logger
from torch.utils.data import DataLoader
from torch import nn

from pose.dataset import pose_dataset
from pose.utils import (
    collate_fn,
    geodesic_distance,
    relative_pose_error,
    relative_pose_error_np,
    recall_object,
    aggregate_metrics
)
from pose.model import Mkpts_Reg_Model
from pose.animator import Animator


if os.name == 'nt':
    LM_dataset_path = 'd:/git_project/POPE/data/LM_dataset/'
    LM_dataset_json_path = 'd:/git_project/POPE/data/pairs/LINEMOD-test.json'
    LM_dataset_points_path = 'd:/git_project/POPE/data/LM_dataset-points/'

    onepose_path = 'd:/git_project/POPE/data/onepose/'
    onepose_json_path = 'd:/git_project/POPE/data/pairs/Onepose-test.json'
    onepose_points_path = 'd:/git_project/POPE/data/onepose-points/'

    onepose_plusplus_path = 'd:/git_project/POPE/data/onepose_plusplus/'
    onepose_plusplus_json_path = 'd:/git_project/POPE/data/pairs/OneposePlusPlus-test.json'
    onepose_plusplus_points_path = 'd:/git_project/POPE/data/onepose_plusplus-points/'

    ycbv_path = 'd:/git_project/POPE/data/ycbv/'
    ycbv_json_path = 'd:/git_project/POPE/data/pairs/YCB-VIDEO-test.json'
    ycbv_points_path = 'd:/git_project/POPE/data/ycbv-points'
elif os.name == 'posix':
    LM_dataset_path = 'data/LM_dataset/'
    LM_dataset_json_path = 'data/pairs/LINEMOD-test.json'
    LM_dataset_points_path = 'data/LM_dataset-points/'

    onepose_path = 'data/onepose/'
    onepose_json_path = 'data/pairs/Onepose-test.json'
    onepose_points_path = 'data/onepose-points/'

    onepose_plusplus_path = 'data/oneposeplusplus/'
    onepose_plusplus_json_path = 'data/pairs/OneposePlusPlus-test.json'
    onepose_plusplus_points_path = 'data/onepose_plusplus-points/'

    ycbv_path = 'data/ycbv/'
    ycbv_json_path = 'data/pairs/YCB-VIDEO-test.json'
    ycbv_points_path = 'data/ycbv-points'

paths = [
    # ('linemod', LM_dataset_path, LM_dataset_json_path, LM_dataset_points_path),
    # ('onepose', onepose_path, onepose_json_path, onepose_points_path),
    ('onepose_plusplus', onepose_plusplus_path, onepose_plusplus_json_path, onepose_plusplus_points_path),
    # ('ycbv', ycbv_path, ycbv_json_path, ycbv_points_path),
]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = pose_dataset(paths)
mkpts_max_len, mkpts_sum_len = dataset.get_mkpts_info()

 56%|█████▌    | 5/9 [00:05<00:04,  1.15s/it]

d:/git_project/POPE/data/onepose_plusplus-points/0706-teabox-box\mkpts0\820.png-761.png.txt does not exist


 89%|████████▉ | 8/9 [00:08<00:01,  1.10s/it]

d:/git_project/POPE/data/onepose_plusplus-points/0712-insta-others\mkpts0\1570.png-125.png.txt does not exist


100%|██████████| 9/9 [00:10<00:00,  1.15s/it]

d:/git_project/POPE/data/onepose_plusplus-points/0712-insta-others\mkpts0\1605.png-185.png.txt does not exist
d:/git_project/POPE/data/onepose_plusplus-points/0712-insta-others\mkpts0\1628.png-210.png.txt does not exist


In [2]:
random.seed(20231223)
torch.manual_seed(20231223)
torch.cuda.manual_seed(20231223)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [3]:
num_sample = 300
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True, collate_fn=collate_fn(num_sample))
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True, drop_last=True, collate_fn=collate_fn(num_sample))

In [4]:
# linemod
# net = torch.load('./weights/linemod-relative_r-gt_t-quat-300-2024-03-08-16-56-16-0.2183.pth').to(device)
# onepose
# net = torch.load('./weights/onepose-relative_r-gt_t-6d-300-2024-03-02-16-10-39-0.2373.pth').to(device)
# onepose++
net = torch.load('./weights/relative_r-gt_t-6d-300-2024-03-12-11-34-40-0.2326.pth').to(device)
# ycbv
# net = torch.load('./weights/ycbv-relative_r-gt_t-6d-300-2024-03-02-17-01-40-6281.5225.pth').to(device)

net.eval()

Mkpts_Reg_Model(
  (embedding): Embedding()
  (transformerlayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=76, out_features=76, bias=True)
    )
    (linear1): Linear(in_features=76, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=76, bias=True)
    (norm1): LayerNorm((76,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((76,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=76, out_features=76, bias=True)
        )
        (linear1): Linear(in_features=76, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inp

In [5]:
id2name_dict = {
    1: "ape",
    2: "benchvise",
    4: "camera",
    5: "can",
    6: "cat",
    8: "driller",
    9: "duck",
    10: "eggbox",
    11: "glue",
    12: "holepuncher",
    13: "iron",
    14: "lamp",
    15: "phone",
}

ycbv_dict = {
    1: 'one',
    2: 'two',
    3: 'three',
    4: 'four',
    5: 'five',
    6: 'six',
    7: 'seven',
    8: 'eight',
    9: 'nine',
    10: 'ten',
}

In [6]:
# linemod
ape_data = []
benchvise_data = []
camera_data = []
can_data = []
cat_data = []
driller_data = []
duck_data = []
eggbox_data = []
glue_data = []
holepuncher_data = []
iron_data = []
lamp_data = []
phone_data = []
# onepose
aptamil_data = []
jzhg_data = []
minipuff_data = []
hlyormosiapie_data = []
brownhouse_data = []
oreo_data = []
mfmilkcake_data = []
diycookies_data = []
taipingcookies_data = []
tee_data = []
# onepose++
toyrobot_data = []
yellowduck_data = []
sheep_data = []
fakebanana_data = []
teabox_data = []
orange_data = []
greenteapot_data = []
lecreusetcup_data = []
insta_data = []
# ycbv
one_data = []
two_data = []
three_data = []
four_data = []
five_data = []
six_data = []
seven_data = []
eight_data = []
nine_data = []
ten_data = []

all_data = {
    # linemod
    'ape_data': ape_data,
    'benchvise_data': benchvise_data,
    'camera_data': camera_data,
    'can_data': can_data,
    'cat_data': cat_data,
    'driller_data': driller_data,
    'duck_data': duck_data,
    'eggbox_data': eggbox_data,
    'glue_data': glue_data,
    'holepuncher_data': holepuncher_data,
    'iron_data': iron_data,
    'lamp_data': lamp_data,
    'phone_data': phone_data,
    # onepose
    'aptamil_data': aptamil_data,
    'jzhg_data': jzhg_data,
    'minipuff_data': minipuff_data,
    'hlyormosiapie_data': hlyormosiapie_data,
    'brownhouse_data': brownhouse_data,
    'oreo_data': oreo_data,
    'mfmilkcake_data': mfmilkcake_data,
    'diycookies_data': diycookies_data,
    'taipingcookies_data': taipingcookies_data,
    'tee_data': tee_data,
    # onepose++
    'toyrobot_data': toyrobot_data,
    'yellowduck_data': yellowduck_data,
    'sheep_data': sheep_data,
    'fakebanana_data': fakebanana_data,
    'teabox_data': teabox_data,
    'orange_data': orange_data,
    'greenteapot_data': greenteapot_data,
    'lecreusetcup_data': lecreusetcup_data,
    'insta_data': insta_data,
    # ycbv
    'one_data': one_data,
    'two_data': two_data,
    'three_data': three_data,
    'four_data': four_data,
    'five_data': five_data,
    'six_data': six_data,
    'seven_data': seven_data,
    'eight_data': eight_data,
    'nine_data': nine_data,
    'ten_data': ten_data,
}

In [7]:
linemod_type = ['ape_data', 'benchvise_data', 'camera_data', 'can_data', 'cat_data', 'driller_data', 'duck_data', 'eggbox_data', 'glue_data', 'holepuncher_data', 'iron_data', 'lamp_data', 'phone_data']
onepose_type = ['aptamil_data', 'jzhg_data', 'minipuff_data', 'hlyormosiapie_data', 'brownhouse_data', 'oreo_data', 'mfmilkcake_data', 'diycookies_data', 'taipingcookies_data', 'tee_data']
oneposeplusplus_type = ['toyrobot_data', 'yellowduck_data', 'sheep_data', 'fakebanana_data', 'teabox_data', 'orange_data', 'greenteapot_data', 'lecreusetcup_data', 'insta_data']
ycbv_type = ['one_data', 'two_data', 'three_data', 'four_data', 'five_data', 'six_data', 'seven_data', 'eight_data', 'nine_data', 'ten_data']

In [8]:
for i, batch in enumerate(test_dataloader):
    for data in batch:
        if 'lm' in data['name']:
            all_data[f"{id2name_dict[int(data['name'][2:])]}_data"].append(data)
        else:
            if data['name'] in ('12345678910'):
                all_data[f"{ycbv_dict[int(data['name'])]}_data"].append(data)
            else:
                all_data[f"{data['name']}_data"].append(data)

empty_keys = []
for key in all_data.keys():
    if len(all_data[key]) == 0:
        empty_keys.append(key)

for key in empty_keys:
    all_data.pop(key)

for key in all_data.keys():
    print(key, len(all_data[key]))

print('len(all_data):', len(all_data))

toyrobot_data 57
yellowduck_data 66
sheep_data 52
fakebanana_data 48
teabox_data 65
orange_data 75
greenteapot_data 40
lecreusetcup_data 47
insta_data 94
len(all_data): 9


In [9]:
res_table = []

model_type = 'relative_r-gt_t'

for key in all_data.keys():
    if key in linemod_type:
        logger.info(f"LINEMOD: {key}")
    elif key in onepose_type:
        logger.info(f"ONEPOSE: {key}")
    elif key in oneposeplusplus_type:
        logger.info(f"ONEPOSE++: {key}")
    elif key in ycbv_type:
        logger.info(f"YCBV: {key}")
    metrics = dict()
    metrics.update({'R_errs':[], 't_errs':[], 'inliers':[], "identifiers":[]})
    recall_image, all_image = 0, 0
    for item in all_data[key]:
        all_image += 1
        K0 = item['K0']
        K1 = item['K1']
        pose0 = item['pose0']
        pose1 = item['pose1']
        pre_bbox = item['pre_bbox']
        gt_bbox = item['gt_bbox']
        mkpts0 = item['mkpts0']
        mkpts1 = item['mkpts1']
        pre_K = item['pre_K']
        name = item['name']
        pair_name = item['pair_name']
        # linemod
        if 'lm' in name:
            name = id2name_dict[int(name[2:])]
        # ycbv
        if name in '12345678910':
            name = ycbv_dict[int(name)]

        if name not in key:
            print(f'name: {name}, key: {key}')
            continue

        is_recalled = recall_object(pre_bbox, gt_bbox)

        recall_image = recall_image + int(is_recalled > 0.5)

        batch_mkpts0 = torch.from_numpy(mkpts0).unsqueeze(0).float().to(device)
        batch_mkpts1 = torch.from_numpy(mkpts1).unsqueeze(0).float().to(device)
        pre_t, pre_rot = net(batch_mkpts0, batch_mkpts1)
        pre_t = pre_t.squeeze(0).detach().cpu().numpy()
        pre_rot = pre_rot.squeeze(0).detach().cpu().numpy()

        if model_type == 'gt':
            t_err, R_err = relative_pose_error_np(pose1, pre_rot, pre_t, ignore_gt_t_thr=0.0)
        elif model_type == 'relative':
            relative_pose = np.matmul(pose1, np.linalg.inv(pose0))
            t_err, R_err = relative_pose_error_np(relative_pose, pre_rot, pre_t, ignore_gt_t_thr=0.0)
        elif model_type == 'relative_r-gt_t':
            relative_pose = np.matmul(pose1, np.linalg.inv(pose0))
            gt_pose = np.zeros_like(pose1)
            gt_pose[:3, :3] = relative_pose[:3, :3]
            gt_pose[:3, 3] = pose1[:3, 3]
            t_err, R_err = relative_pose_error_np(gt_pose, pre_rot, pre_t, ignore_gt_t_thr=0.0)

        metrics['R_errs'].append(R_err)
        metrics['t_errs'].append(t_err)
        metrics['identifiers'].append(pair_name)

    print(f"Acc: {recall_image}/{all_image}")
    val_metrics_4tb = aggregate_metrics(metrics, 5e-4)
    val_metrics_4tb["AP50"] = recall_image / all_image
    logger.info('\n' + pprint.pformat(val_metrics_4tb))
    res_table.append([f"{name}"] + list(val_metrics_4tb.values()))

2024-03-12 11:35:50.975 | INFO     | __main__:<module>:11 - ONEPOSE++: toyrobot_data


2024-03-12 11:35:53.246 | INFO     | pose.utils:aggregate_metrics:237 - Aggregating metrics over 57 unique items...
2024-03-12 11:35:53.248 | INFO     | __main__:<module>:70 - 
{'AP50': 0.9824561403508771,
 'R:ACC15': 0.47368421052631576,
 'R:ACC30': 0.9824561403508771,
 'R:auc@15': 0.18344907412571204,
 'R:auc@30': 0.46382076561553714,
 'R:meanErr': 16.348149095431815,
 'R:medianErr': 15.402956107289398,
 't:ACC15': 0.9824561403508771,
 't:ACC30': 1.0,
 't:auc@15': 0.5505204620100249,
 't:auc@30': 0.7749006944590646,
 't:meanErr': 6.90187803329131,
 't:medianErr': 6.669386150658583}
2024-03-12 11:35:53.248 | INFO     | __main__:<module>:11 - ONEPOSE++: yellowduck_data


Acc: 56/57


2024-03-12 11:35:53.882 | INFO     | pose.utils:aggregate_metrics:237 - Aggregating metrics over 66 unique items...
2024-03-12 11:35:53.884 | INFO     | __main__:<module>:70 - 
{'AP50': 1.0,
 'R:ACC15': 0.5606060606060606,
 'R:ACC30': 1.0,
 'R:auc@15': 0.27766620270318065,
 'R:auc@30': 0.5615526463144423,
 'R:meanErr': 13.378617323029673,
 'R:medianErr': 13.22067428206968,
 't:ACC15': 0.9696969696969697,
 't:ACC30': 1.0,
 't:auc@15': 0.5580346370151738,
 't:auc@30': 0.7789683548456792,
 't:meanErr': 6.751760028680323,
 't:medianErr': 6.635839354050449}
2024-03-12 11:35:53.885 | INFO     | __main__:<module>:11 - ONEPOSE++: sheep_data


Acc: 66/66


...

Acc: 37/40


2024-03-12 11:35:57.351 | INFO     | pose.utils:aggregate_metrics:237 - Aggregating metrics over 47 unique items...
2024-03-12 11:35:57.353 | INFO     | __main__:<module>:70 - 
{'AP50': 0.851063829787234,
 'R:ACC15': 0.574468085106383,
 'R:ACC30': 0.9787234042553191,
 'R:auc@15': 0.3026316143294921,
 'R:auc@30': 0.5425793541734923,
 'R:meanErr': 14.309750019734196,
 'R:medianErr': 12.744561110597886,
 't:ACC15': 0.9361702127659575,
 't:ACC30': 0.9787234042553191,
 't:auc@15': 0.5043453458206172,
 't:auc@30': 0.7408876966136321,
 't:meanErr': 7.977139003147025,
 't:medianErr': 7.834804933988915}
2024-03-12 11:35:57.353 | INFO     | __main__:<module>:11 - ONEPOSE++: insta_data


Acc: 40/47


2024-03-12 11:35:58.555 | INFO     | pose.utils:aggregate_metrics:237 - Aggregating metrics over 94 unique items...
2024-03-12 11:35:58.557 | INFO     | __main__:<module>:70 - 
{'AP50': 0.5638297872340425,
 'R:ACC15': 0.4574468085106383,
 'R:ACC30': 0.8936170212765957,
 'R:auc@15': 0.20766420306816186,
 'R:auc@30': 0.4455821454438348,
 'R:meanErr': 18.17409857594756,
 'R:medianErr': 17.01620111775686,
 't:ACC15': 0.925531914893617,
 't:ACC30': 0.9468085106382979,
 't:auc@15': 0.6048325142071836,
 't:auc@30': 0.772483767690792,
 't:meanErr': 7.917751815760798,
 't:medianErr': 4.685330818966257}


Acc: 53/94


In [10]:
for i, key in enumerate(all_data.keys()):
    if i == 1:
        break
    if key in linemod_type:
        print(f"LINEMOD")
    elif key in onepose_type:
        print(f"ONEPOSE")
    elif key in oneposeplusplus_type:
        print(f"ONEPOSE++")
    elif key in ycbv_type:
        print(f"YCBV")

from tabulate import tabulate
headers = ["Category"] + list(val_metrics_4tb.keys())
all_data = np.array(res_table)[:, 1:].astype(np.float32)
res_table.append(["Avg"] + all_data.mean(0).tolist())
print(tabulate(res_table, headers=headers, tablefmt='fancy_grid'))

ONEPOSE++
╒══════════════╤════════════╤════════════╤═══════════╤═══════════╤═══════════════╤═════════════╤════════════╤════════════╤═══════════╤═══════════╤═══════════════╤═════════════╤══════════╕
│ Category     │   R:auc@15 │   R:auc@30 │   R:ACC15 │   R:ACC30 │   R:medianErr │   R:meanErr │   t:auc@15 │   t:auc@30 │   t:ACC15 │   t:ACC30 │   t:medianErr │   t:meanErr │     AP50 │
╞══════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════════╪═════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════════╪═════════════╪══════════╡
│ toyrobot     │   0.183449 │   0.463821 │  0.473684 │  0.982456 │       15.403  │     16.3481 │   0.55052  │   0.774901 │  0.982456 │  1        │       6.66939 │     6.90188 │ 0.982456 │
├──────────────┼────────────┼────────────┼───────────┼───────────┼───────────────┼─────────────┼────────────┼────────────┼───────────┼───────────┼───────────────┼─────────────┼──────────┤
│ yellowduck   │   0.277666 │   0.561553 │  0.5606